# Upstream SDK Core Demo

This notebook demonstrates the core functionality of the Upstream SDK for managing environmental monitoring campaigns, stations, and data upload.

## Overview

The Upstream SDK provides a modern, type-safe interface for:
- 🏕️ **Campaign Management**: Creating and managing monitoring campaigns
- 📡 **Station Management**: Setting up monitoring stations with sensors
- 📊 **Data Management**: Uploading sensor data and measurements
- 📈 **Data Retrieval**: Querying and analyzing uploaded data

## Features Demonstrated

- Authentication and client initialization
- Campaign creation and management
- Station setup and configuration
- Data upload with CSV file handling
- Sensor management and statistics
- Error handling and validation

## Prerequisites

- Valid Upstream account credentials
- Python 3.7+ environment
- Required packages installed (see requirements)

## Related Notebooks

- **UpstreamSDK_CKAN_Demo.ipynb**: CKAN integration and data publishing
- **UpstreamSDK_Demo.ipynb**: Original combined demo (deprecated)

## Installation and Setup

In [1]:
# Install required packages
!pip install -e .

# Import required libraries
import os
import json
import getpass
from pathlib import Path
from datetime import datetime
from typing import Dict, Any, Optional, List

# Import Upstream SDK modules
from upstream.client import UpstreamClient
from upstream.campaigns import CampaignManager
from upstream.stations import StationManager
from upstream.exceptions import APIError, ValidationError
from upstream.auth import AuthManager

Obtaining file:///Users/mosorio/repos/tacc/upstream/sdk
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for upstream-sdk (pyproject.toml) ... done
  Created wheel for upstream-sdk: filename=upstream_sdk-1.0.0-0.editable-py3-none-any.whl size=8003 sha256=9e72eeb2ef7e3f035135ab71e2bd7b73082fa44a9d5b7ec581c7ae928763476c
  Stored in directory: /private/var/folders/qn/xpsy3ssx5hbbb_ndr2sbt5w80000gn/T/pip-ephem-wheel-cache-tpmw2cpi/wheels/47/dc/ae/1a3abd774032839edac85dcd8bb9739031dd6ccef29fca9667
Successfully built upstream-sdk
  Attempting uninstall: upstream-sdk
    Found existing installation: upstream-sdk 1.0.0
    Uninstalling upstream-sdk-1.0.0:
      Successfully uninstalled upstream-sdk-1.0.0

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## 1. Authentication and Client Setup

First, let's authenticate with the Upstream API and set up our client instances.

**Note**: Update the `BASE_URL` according to your environment:
- Development: `http://localhost:8000`
- Production: `https://upstream-dso.tacc.utexas.edu/dev`

In [2]:
# Configuration
BASE_URL = "https://upstream-dso.tacc.utexas.edu/dev"
# For local development, uncomment the line below:
BASE_URL = 'http://localhost:8000'

# Get credentials securely
print("Please enter your Upstream credentials:")
username = input("Username: ")
password = getpass.getpass("Password: ")

# Initialize client
try:
    client = UpstreamClient(
        username=username,
        password=password,
        base_url=BASE_URL
    )

    # Test authentication
    if client.authenticate():
        print("✅ Authentication successful!")
        print(f"🔗 Connected to: {BASE_URL}")
    else:
        print("❌ Authentication failed!")
        raise Exception("Authentication failed")

except Exception as e:
    print(f"❌ Setup error: {e}")
    raise

Please enter your Upstream credentials:
✅ Authentication successful!
🔗 Connected to: http://localhost:8000


## 2. Campaign Management

Campaigns are the top-level organizational unit in Upstream. They represent a specific monitoring project with defined goals, timeframes, and responsible parties.

### Creating a New Campaign

In [6]:
# Initialize campaign manager
from upstream_api_client.models import CampaignsIn
campaign_manager = CampaignManager(client.auth_manager)

# Create campaign request with all required fields
campaign_request = CampaignsIn(
    name="Environmental Monitoring Core Demo 2024",
    description="Core functionality demonstration for SDK campaign and station management",
    contact_name="Dr. Jane Smith",
    contact_email="jane.smith@example.edu",
    allocation="TACC",
    start_date=datetime.now(),
    end_date=datetime.now().replace(year=datetime.now().year + 1)
)

# Create a new campaign
print("📊 Creating new campaign...")
try:
    campaign = campaign_manager.create(campaign_request)
    print(f"✅ Campaign created successfully!")
    print(f"   ID: {campaign.id}")
    campaign_id = campaign.id

except ValidationError as e:
    print(f"❌ Validation error: {e}")
except APIError as e:
    print(f"❌ API error: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

📊 Creating new campaign...
✅ Campaign created successfully!
   ID: 611


### Listing Existing Campaigns

In [7]:
# List existing campaigns
print("📋 Listing existing campaigns...")
try:
    campaigns = campaign_manager.list(limit=10)
    print(f"Found {campaigns.total} campaigns:")

    for camp in campaigns.items[:5]:  # Show first 5
        print(f"  • {camp.id}: {camp.name}")
        print(f"    Description: {camp.description[:100]}...")
        print(f"    Contact: {camp.contact_name} ({camp.contact_email})")
        print(f"    Duration: {camp.start_date.strftime('%Y-%m-%d')} to {camp.end_date.strftime('%Y-%m-%d')}")
        print()

except Exception as e:
    print(f"❌ Error listing campaigns: {e}")

📋 Listing existing campaigns...
Found 2 campaigns:
  • 1: Test Campaign 2024
    Description: A test campaign for development purposes...
    Contact: John Doe (john.doe@example.com)
    Duration: 2024-01-01 to 2024-12-31

  • 2: Weather Station Network
    Description: Network of weather stations across Texas...
    Contact: Jane Smith (jane.smith@example.com)
    Duration: 2024-03-01 to 2025-02-28



### Getting Campaign Details

In [8]:
# Get detailed campaign information
print(f"📋 Getting campaign details for ID: {campaign_id}")
try:
    campaign_details = campaign_manager.get(str(campaign_id))

    print(f"\n📊 Campaign Details:")
    print(f"  Name: {campaign_details.name}")
    print(f"  Description: {campaign_details.description}")
    print(f"  Contact: {campaign_details.contact_name} ({campaign_details.contact_email})")
    print(f"  Allocation: {campaign_details.allocation}")
    print(f"  Start Date: {campaign_details.start_date}")
    print(f"  End Date: {campaign_details.end_date}")
    print(f"  Created: {campaign_details.created_at}")
    print(f"  Updated: {campaign_details.updated_at}")

except Exception as e:
    print(f"❌ Error getting campaign details: {e}")

📋 Getting campaign details for ID: 611

📊 Campaign Details:
  Name: Environmental Monitoring Core Demo 2024
  Description: Core functionality demonstration for SDK campaign and station management
  Contact: Dr. Jane Smith (jane.smith@example.edu)
  Allocation: TACC
  Start Date: 2025-07-17 09:33:24.269422
  End Date: 2026-07-17 09:33:24.269450
❌ Error getting campaign details: 'GetCampaignResponse' object has no attribute 'created_at'


## 3. Station Management

Stations represent physical monitoring locations within a campaign. Each station can have multiple sensors and collect various types of environmental data.

### Creating a New Station

In [11]:
# Initialize station manager
station_manager = StationManager(client.auth_manager)
from upstream_api_client.models import StationCreate

# Create station with comprehensive information
new_station = StationCreate(
    name="Downtown Air Quality Monitor",
    description="Air quality monitoring station in downtown Austin with PM2.5, PM10, and meteorological sensors",
    contact_name="Dr. Jane Smith",
    contact_email="jane.smith@example.edu",
    start_date=datetime.now(),
)

# Create a new station
print("📍 Creating new monitoring station...")
try:
    station = station_manager.create(campaign_id=str(campaign_id), station_create=new_station)
    print(f"✅ Station created successfully!")
    print(f"   ID: {station.id}")
    station_id = station.id

except ValidationError as e:
    print(f"❌ Validation error: {e}")
except APIError as e:
    print(f"❌ API error: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

📍 Creating new monitoring station...
✅ Station created successfully!
   ID: 482


### Listing Stations in Campaign

In [12]:
# List stations in the campaign
print(f"📋 Listing stations in campaign {campaign_id}...")
try:
    stations = station_manager.list(campaign_id=str(campaign_id))
    print(f"Found {stations.total} stations:")

    for station in stations.items:
        print(f"  • {station.id}: {station.name}")
        print(f"    Description: {station.description[:80]}...")
        print(f"    Contact: {station.contact_name}")
        print(f"    Start Date: {station.start_date}")
        print()

except Exception as e:
    print(f"❌ Error listing stations: {e}")

📋 Listing stations in campaign 611...
Found 3 stations:
  • 480: Downtown Air Quality Monitor
    Description: Air quality monitoring station in downtown Austin with PM2.5, PM10, and meteorol...
    Contact: None
    Start Date: None

  • 481: Downtown Air Quality Monitor
    Description: Air quality monitoring station in downtown Austin with PM2.5, PM10, and meteorol...
    Contact: None
    Start Date: None

  • 482: Downtown Air Quality Monitor
    Description: Air quality monitoring station in downtown Austin with PM2.5, PM10, and meteorol...
    Contact: None
    Start Date: None



## 4. Data Upload and Management

The Upstream SDK supports CSV-based data upload with separate files for sensor definitions and measurements.

### Creating Sample Data Files

In [14]:
# Create sample data directory
data_dir = Path("sample_data")
data_dir.mkdir(exist_ok=True)

# Create comprehensive sensors CSV with various sensor types
sensors_csv = data_dir / "sensors.csv"
sensors_data = """alias,variablename,units,postprocess,postprocessscript
temp_01,Air Temperature,°C,false,
humidity_01,Relative Humidity,%,false,
pressure_01,Atmospheric Pressure,hPa,false,
pm25_01,PM2.5 Concentration,μg/m³,true,pm25_calibration
pm10_01,PM10 Concentration,μg/m³,true,pm10_calibration
wind_speed,Wind Speed,m/s,false,
wind_dir,Wind Direction,degrees,false,
co2_01,CO2 Concentration,ppm,false,"""

with open(sensors_csv, 'w') as f:
    f.write(sensors_data)

# Create sample measurements CSV with realistic data patterns
measurements_csv = data_dir / "measurements.csv"
measurements_data = """collectiontime,Lat_deg,Lon_deg,temp_01,humidity_01,pressure_01,pm25_01,pm10_01,wind_speed,wind_dir,co2_01
2024-01-15T10:00:00,30.2672,-97.7431,22.5,68.2,1013.25,15.2,25.8,3.2,45,420
2024-01-15T10:05:00,30.2672,-97.7431,22.7,67.8,1013.20,14.8,24.5,3.5,52,425
2024-01-15T10:10:00,30.2672,-97.7431,22.9,67.5,1013.15,16.1,26.2,3.1,48,418
2024-01-15T10:15:00,30.2672,-97.7431,23.1,67.2,1013.10,15.5,25.1,2.8,41,430
2024-01-15T10:20:00,30.2672,-97.7431,23.3,66.9,1013.05,14.9,24.8,3.3,55,422
2024-01-15T10:25:00,30.2672,-97.7431,23.5,66.5,1013.00,15.7,26.0,3.0,47,428
2024-01-15T10:30:00,30.2672,-97.7431,23.7,66.2,1012.95,16.2,26.5,3.4,50,415
2024-01-15T10:35:00,30.2672,-97.7431,23.9,65.9,1012.90,15.3,25.3,2.9,43,435
2024-01-15T10:40:00,30.2672,-97.7431,24.1,65.6,1012.85,14.6,24.2,3.6,58,420
2024-01-15T10:45:00,30.2672,-97.7431,24.3,65.3,1012.80,15.8,25.9,3.2,46,425
2024-01-15T10:50:00,30.2672,-97.7431,24.5,65.0,1012.75,16.5,26.8,3.1,49,412
2024-01-15T10:55:00,30.2672,-97.7431,24.7,64.8,1012.70,15.1,24.9,3.7,61,438"""

with open(measurements_csv, 'w') as f:
    f.write(measurements_data)

print(f"📁 Sample data files created:")
print(f"  • Sensors: {sensors_csv} ({sensors_csv.stat().st_size} bytes)")
print(f"  • Measurements: {measurements_csv} ({measurements_csv.stat().st_size} bytes)")


📁 Sample data files created:
  • Sensors: sample_data/sensors.csv (395 bytes)
  • Measurements: sample_data/measurements.csv (1017 bytes)


### Uploading Data to Station

In [15]:
# Upload CSV data to the station
print(f"📤 Uploading sensor data to station {station_id}...")
try:
    upload_result = client.upload_csv_data(
        campaign_id=campaign_id,
        station_id=station_id,
        sensors_file=sensors_csv,
        measurements_file=measurements_csv
    )

    print(f"✅ Data uploaded successfully!")
    print(f"\n📊 Upload Summary:")
    response = upload_result['response']
    print(f"  • Sensors processed: {response.get('Total sensors processed', 'N/A')}")
    print(f"  • Measurements added: {response.get('Total measurements added to database', 'N/A')}")
    print(f"  • Processing time: {response.get('Data Processing time', 'N/A')}")
    print(f"  • Sensors file stored: {response.get('uploaded_file_sensors stored in memory', 'N/A')}")
    print(f"  • Measurements file stored: {response.get('uploaded_file_measurements stored in memory', 'N/A')}")

except Exception as e:
    print(f"❌ Upload error: {e}")

📤 Uploading sensor data to station 482...
✅ Data uploaded successfully!

📊 Upload Summary:
  • Sensors processed: 8
  • Measurements added: 96
  • Processing time: 0.2 seconds.
  • Sensors file stored: True
  • Measurements file stored: True


## 5. Sensor Management and Statistics

After uploading data, we can explore the sensors and their measurement statistics.

### Listing Sensors on Station

In [16]:
# List all sensors on the station
print(f"📡 Listing all sensors on station {station_id}...")
try:
    sensors = client.sensors.list(
        campaign_id=campaign_id,
        station_id=station_id
    )

    print(f"Found {len(sensors.items)} sensors:")
    for sensor in sensors.items:
        print(f"  • {sensor.alias} ({sensor.variablename})")
        print(f"    Units: {sensor.units}")
        print(f"    Post-process: {sensor.postprocess}")
        if sensor.postprocessscript:
            print(f"    Post-process script: {sensor.postprocessscript}")
        print()

except Exception as e:
    print(f"❌ Error listing sensors: {e}")

📡 Listing all sensors on station 482...
Found 8 sensors:
  • temp_01 (Air Temperature)
    Units: °C
    Post-process: False

  • humidity_01 (Relative Humidity)
    Units: %
    Post-process: False

  • pressure_01 (Atmospheric Pressure)
    Units: hPa
    Post-process: False

  • pm25_01 (PM2.5 Concentration)
    Units: μg/m³
    Post-process: True
    Post-process script: pm25_calibration

  • pm10_01 (PM10 Concentration)
    Units: μg/m³
    Post-process: True
    Post-process script: pm10_calibration

  • wind_speed (Wind Speed)
    Units: m/s
    Post-process: False

  • wind_dir (Wind Direction)
    Units: degrees
    Post-process: False

  • co2_01 (CO2 Concentration)
    Units: ppm
    Post-process: False



### Sensor Statistics Analysis

In [17]:
# Analyze sensor statistics
print(f"📈 Analyzing sensor statistics...")
try:
    # Get first sensor for detailed analysis
    if sensors.items:
        sensor = sensors.items[0]
        stats = sensor.statistics

        print(f"\n📊 Detailed Statistics for {sensor.alias} ({sensor.variablename}):")
        print(f"  • Total measurements: {stats.count}")
        print(f"  • Value range: {stats.min_value:.2f} - {stats.max_value:.2f} {sensor.units}")
        print(f"  • Average value: {stats.avg_value:.2f} {sensor.units}")
        print(f"  • Standard deviation: {stats.stddev_value:.3f}")
        print(f"  • 95th percentile: {stats.percentile_95:.2f} {sensor.units}")
        print(f"  • 99th percentile: {stats.percentile_99:.2f} {sensor.units}")
        print(f"  • First measurement: {stats.first_measurement_collectiontime}")
        print(f"  • Last measurement: {stats.last_measurement_time}")
        print(f"  • Last value: {stats.last_measurement_value:.2f} {sensor.units}")
        print(f"  • Statistics updated: {stats.stats_last_updated}")

        # Show statistics for all sensors in summary
        print(f"\n📋 Summary Statistics for All Sensors:")
        for sensor in sensors.items:
            stats = sensor.statistics
            print(f"  • {sensor.alias}: {stats.count} measurements, avg={stats.avg_value:.2f} {sensor.units}")

    else:
        print("  No sensors found for analysis")

except Exception as e:
    print(f"❌ Error analyzing sensor statistics: {e}")

📈 Analyzing sensor statistics...

📊 Detailed Statistics for temp_01 (Air Temperature):
  • Total measurements: 12
  • Value range: 22.50 - 24.70 °C
  • Average value: 23.60 °C
  • Standard deviation: 0.721
  • 95th percentile: 24.59 °C
  • 99th percentile: 24.68 °C
  • First measurement: 2024-01-15 10:00:00+00:00
  • Last measurement: 2024-01-15 10:55:00+00:00
  • Last value: 24.70 °C
  • Statistics updated: 2025-07-17 13:35:07.526184+00:00

📋 Summary Statistics for All Sensors:
  • temp_01: 12 measurements, avg=23.60 °C
  • humidity_01: 12 measurements, avg=66.41 %
  • pressure_01: 12 measurements, avg=1012.98 hPa
  • pm25_01: 12 measurements, avg=15.47 μg/m³
  • pm10_01: 12 measurements, avg=25.50 μg/m³
  • wind_speed: 12 measurements, avg=3.23 m/s
  • wind_dir: 12 measurements, avg=49.58 degrees
  • co2_01: 12 measurements, avg=424.00 ppm


## 6. Data Retrieval and Analysis

Let's retrieve and analyze the campaign data we've created.

### Campaign Summary

In [19]:
# Get comprehensive campaign summary
print(f"📊 Comprehensive Campaign Summary for ID {campaign_id}:")
try:
    campaign_details = campaign_manager.get(str(campaign_id))
    stations_list = station_manager.list(campaign_id=str(campaign_id))

    print(f"\n📋 Campaign Information:")
    print(f"   Name: {campaign_details.name}")
    print(f"   Description: {campaign_details.description}")
    print(f"   Contact: {campaign_details.contact_name} ({campaign_details.contact_email})")
    print(f"   Allocation: {campaign_details.allocation}")
    print(f"   Start Date: {campaign_details.start_date}")
    print(f"   End Date: {campaign_details.end_date}")

    print(f"\n📍 Station Summary ({stations_list.total} total):")
    for station in stations_list.items:
        print(f"   • {station.name} (ID: {station.id})")
        print(f"     Description: {station.description}")
        print(f"     Contact: {station.contact_name}")
        print(f"     Start Date: {station.start_date}")

        # Get sensor count for this station
        try:
            station_sensors = client.sensors.list(
                campaign_id=campaign_id,
                station_id=station.id
            )
            print(f"     Sensors: {len(station_sensors.items)}")

            # Show total measurements across all sensors
            total_measurements = sum(s.statistics.count for s in station_sensors.items)
            print(f"     Total measurements: {total_measurements}")

        except Exception as e:
            print(f"     Error getting sensor info: {e}")

        print()

except Exception as e:
    print(f"❌ Error getting campaign summary: {e}")

📊 Comprehensive Campaign Summary for ID 611:

📋 Campaign Information:
   Name: Environmental Monitoring Core Demo 2024
   Description: Core functionality demonstration for SDK campaign and station management
   Contact: Dr. Jane Smith (jane.smith@example.edu)
   Allocation: TACC
   Start Date: 2025-07-17 09:33:24.269422
   End Date: 2026-07-17 09:33:24.269450

📍 Station Summary (3 total):
   • Downtown Air Quality Monitor (ID: 480)
     Description: Air quality monitoring station in downtown Austin with PM2.5, PM10, and meteorological sensors
     Contact: None
     Start Date: None
     Sensors: 0
     Total measurements: 0

   • Downtown Air Quality Monitor (ID: 481)
     Description: Air quality monitoring station in downtown Austin with PM2.5, PM10, and meteorological sensors
     Contact: None
     Start Date: None
     Sensors: 0
     Total measurements: 0

   • Downtown Air Quality Monitor (ID: 482)
     Description: Air quality monitoring station in downtown Austin with PM2.5, 

## 7. Error Handling and Validation

Let's demonstrate proper error handling and validation patterns.

### Testing Validation Errors

In [20]:
# Test validation and error handling
print("🧪 Testing validation and error handling...")

# Test API errors with non-existent resources
print("\n1. Testing API errors with non-existent campaign:")
try:
    nonexistent_campaign = campaign_manager.get("999999")
    print(f"   ❌ Should have failed but got: {nonexistent_campaign}")
except APIError as e:
    print(f"   ✅ Caught expected API error: {e}")
except Exception as e:
    print(f"   ⚠️  Caught unexpected error: {e}")

# Test API errors with non-existent station
print("\n2. Testing API errors with non-existent station:")
try:
    nonexistent_station = station_manager.get(
        station_id="999999",
        campaign_id=str(campaign_id)
    )
    print(f"   ❌ Should have failed but got: {nonexistent_station}")
except APIError as e:
    print(f"   ✅ Caught expected API error: {e}")
except Exception as e:
    print(f"   ⚠️  Caught unexpected error: {e}")

# Test network/connection errors
print("\n3. Testing connection handling:")
try:
    # Test with invalid campaign format
    invalid_campaign = campaign_manager.get("invalid-format")
    print(f"   ❌ Should have failed but got: {invalid_campaign}")
except (APIError, ValidationError) as e:
    print(f"   ✅ Caught expected error: {e}")
except Exception as e:
    print(f"   ⚠️  Caught unexpected error: {e}")

print("\n✅ Error handling tests completed")

🧪 Testing validation and error handling...

1. Testing API errors with non-existent campaign:
   ✅ Caught expected API error: Campaign not found: 999999

2. Testing API errors with non-existent station:
   ✅ Caught expected API error: Station not found: 999999

3. Testing connection handling:
   ✅ Caught expected error: Invalid campaign ID format: invalid-format

✅ Error handling tests completed


## 8. Best Practices and Tips

### Data Upload Best Practices

In [21]:
# Demonstrate best practices for data validation
print("💡 Data Upload Best Practices:")

# Check file sizes before upload
print("\n1. File Size Validation:")
sensors_size = sensors_csv.stat().st_size
measurements_size = measurements_csv.stat().st_size
print(f"   • Sensors file: {sensors_size:,} bytes")
print(f"   • Measurements file: {measurements_size:,} bytes")

# Recommend file size limits
MAX_FILE_SIZE = 50 * 1024 * 1024  # 50MB
if sensors_size > MAX_FILE_SIZE or measurements_size > MAX_FILE_SIZE:
    print(f"   ⚠️  Warning: Files exceed recommended size limit ({MAX_FILE_SIZE:,} bytes)")
    print(f"   💡 Consider splitting large files into smaller chunks")
else:
    print(f"   ✅ File sizes are within recommended limits")

# Data validation tips
print("\n2. Data Validation Tips:")
print("   • Ensure CSV files have proper headers")
print("   • Check for missing or invalid timestamps")
print("   • Validate sensor aliases match between files")
print("   • Use consistent units and formatting")
print("   • Include geographic coordinates (Lat_deg, Lon_deg)")

# Performance tips
print("\n3. Performance Tips:")
print("   • Upload data in chronological order")
print("   • Use batch uploads for large datasets")
print("   • Monitor upload progress for large files")
print("   • Consider data compression for large files")
print("   • Use post-processing scripts for data quality checks")

💡 Data Upload Best Practices:

1. File Size Validation:
   • Sensors file: 395 bytes
   • Measurements file: 1,017 bytes
   ✅ File sizes are within recommended limits

2. Data Validation Tips:
   • Ensure CSV files have proper headers
   • Check for missing or invalid timestamps
   • Validate sensor aliases match between files
   • Use consistent units and formatting
   • Include geographic coordinates (Lat_deg, Lon_deg)

3. Performance Tips:
   • Upload data in chronological order
   • Use batch uploads for large datasets
   • Monitor upload progress for large files
   • Consider data compression for large files
   • Use post-processing scripts for data quality checks


## 9. Cleanup

Let's clean up temporary files and log out properly.

In [22]:
# Clean up temporary files
print("🧹 Cleaning up temporary files...")
try:
    if data_dir.exists():
        import shutil
        shutil.rmtree(data_dir)
        print(f"   ✅ Removed {data_dir}")
    else:
        print(f"   ℹ️  Directory {data_dir} does not exist")
except Exception as e:
    print(f"   ❌ Error cleaning up: {e}")

# Logout
print("\n👋 Logging out...")
try:
    client.logout()
    print("   ✅ Logged out successfully")
except Exception as e:
    print(f"   ❌ Logout error: {e}")

print("\n🎉 Core demo completed successfully!")
print("\n📚 Next Steps:")
print("   • Run UpstreamSDK_CKAN_Demo.ipynb for CKAN integration")
print("   • Explore real-time data streaming")
print("   • Set up automated data processing pipelines")
print("   • Develop custom visualization dashboards")

🧹 Cleaning up temporary files...
   ✅ Removed sample_data

👋 Logging out...
   ✅ Logged out successfully

🎉 Core demo completed successfully!

📚 Next Steps:
   • Run UpstreamSDK_CKAN_Demo.ipynb for CKAN integration
   • Explore real-time data streaming
   • Set up automated data processing pipelines
   • Develop custom visualization dashboards


## Summary

This notebook demonstrated the core functionality of the Upstream SDK:

✅ **Authentication** - Secure login to the Upstream platform  
✅ **Campaign Management** - Creating and managing monitoring campaigns  
✅ **Station Management** - Setting up monitoring stations  
✅ **Data Upload** - Uploading sensor and measurement data via CSV files  
✅ **Sensor Management** - Managing sensors and analyzing statistics  
✅ **Data Retrieval** - Querying and analyzing uploaded data  
✅ **Error Handling** - Proper validation and exception handling  
✅ **Best Practices** - File handling, validation, and performance tips  

## Key Takeaways

- **Campaign Structure**: Campaigns → Stations → Sensors → Measurements
- **Data Format**: CSV files with separate sensor definitions and measurements
- **Authentication**: Secure credential-based authentication
- **Error Handling**: Comprehensive validation and API error handling
- **Statistics**: Automatic calculation of sensor measurement statistics

## Related Documentation

- [Upstream SDK Documentation](https://upstream-sdk.readthedocs.io/)
- [API Reference](https://upstream-dso.tacc.utexas.edu/docs)
- [Environmental Data Standards](https://www.example.com/standards)

---

*This notebook demonstrates the core Upstream SDK functionality. For CKAN integration, see UpstreamSDK_CKAN_Demo.ipynb*